In [1]:
import data, score, context as ctx

model, tokenizer = score.get_model()
short_result = data.get_context_sentences(ctx.context, ctx.context_t_i, ctx.context_a_i)
long_result = data.get_context_sentences(ctx.context_long, ctx.context_long_t_i, ctx.context_long_a_i)

/home/gussuvmi@GU.GU.SE/venv/lib64/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
def get_prior_prob(sentence_dict):
    result = {}
    for key in sentence_dict.keys():
        sents = sentence_dict[key]
        if key == 'fin':
            # just take whatever key, attributes in sentence will be masked anyway for prior probability
            k = list(sentence_dict[key].keys())[0]
            sents = sentence_dict[key][k]
        result[key] = {}
        for sent in sents:
            if sent[3] not in result[key]:
                _, prior_prob = score.get_sentence_probabilities(model, tokenizer, sent[0], sent[1], sent[2])
                result[key][sent[3]] = prior_prob
    return result
s = get_prior_prob(short_result)
l = get_prior_prob(long_result)

In [41]:
import pandas as pd

def get_prior_df(s):
    prior_l = []

    for entity in s['fin'].keys():
        ent_priors = []
        for ethnicity in s.keys():
            ent_priors.append(float(s[ethnicity][entity]))
        prior_l.append(ent_priors)


    df = pd.DataFrame(prior_l, columns=s.keys())
    df.index = s['fin'].keys()
    return df

get_prior_df(s)

,fin,fswe,roma,som,sami,rus
nainen,1.508288e-01,1.317149e-01,2.886688e-03,2.911757e-03,2.502810e-04,7.543596e-03
mies,4.636000e-02,5.346810e-02,2.529281e-04,1.808611e-04,4.794412e-05,1.063527e-03
henkilö,1.439063e-03,3.582885e-02,3.882793e-04,6.128108e-05,2.154109e-05,1.388483e-04
female name,1.182909e-07,1.296816e-06,8.125729e-06,7.845769e-08,1.196418e-08,2.514358e-07
male name,1.014819e-06,1.535878e-07,9.300995e-08,3.472587e-07,1.255568e-07,5.422669e-07


In [42]:
get_prior_df(l)
# TODO can I do one of those correlation things here? association mean vs prior probability

,fin,fswe,roma,som,sami,rus
nainen,0.121355,0.056415,1.283912e-02,1.814083e-02,0.001725,0.057987
mies,0.026863,0.038578,6.369622e-04,7.982593e-04,0.000412,0.007362
henkilö,0.012230,0.026704,2.257069e-03,1.138488e-03,0.000087,0.007057
female name,0.000019,0.000178,7.913087e-05,7.230553e-09,0.000002,0.000013
male name,0.000035,0.000002,2.394042e-07,5.567810e-04,0.000023,0.000071


In [60]:
import numpy as np
# some dumb checks
np.log(0.5 / 0.121355) # 1.415887963966969
np.log(0.2 / 0.121355) # 0.4995972320928141
np.log(0.5 / 1.283912e-02) # 3.6621113383346087
np.log(0.2 / 1.283912e-02) # 2.7458206064604536
np.log(0.002 / 1.283912e-02) # -1.8593495795276376

-1.8593495795276376